In [ ]:
import os
import s3fs
from aicsimageio.writers import OmeZarrWriter
from aicsimageio import AICSImage
from aicsimageio.dimensions import DimensionNames, DEFAULT_CHUNK_DIMS
import numpy

In [ ]:
# set up some initial vars to find our data and where to put it

filepath = "my/path/to/data/file.tif"
output_filename = "my_filename"
output_bucket = "my_bucket"
# aws config
os.environ["AWS_PROFILE"] = "my_creds"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"



In [ ]:
# allow for dask parallelism
from distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4, processes=True, threads_per_worker=1)
client = Client(cluster)
client

In [ ]:
# load our image

chunk_dims= [
    DimensionNames.SpatialY,
    DimensionNames.SpatialX,
    DimensionNames.Samples,
]
img = AICSImage(filepath, chunk_dims=chunk_dims)

# print some data about the image we loaded
scenes = img.scenes
print(scenes)
print(str(len(scenes)))
print(img.channel_names)
print(img.physical_pixel_sizes)



In [ ]:
# construct some per-channel lists to feed in to the writer.
# hardcoding to 9 for now
channel_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0xff00ff, 0x00ffff, 0x880000, 0x008800, 0x000088]

# initialize for writing direct to S3
s3 = s3fs.S3FileSystem(anon=False, config_kwargs={"connect_timeout": 60})

def write_scene(storeroot, scenename, sceneindex, img):
    print(scenename)
    print(storeroot)
    img.set_scene(sceneindex)
    pps = img.physical_pixel_sizes
    cn = img.channel_names

    data = img.get_image_dask_data("TCZYX")
    print(data.shape)
    writer = OmeZarrWriter(storeroot)

    writer.write_image(
        image_data=data,
        image_name=scenename,
        physical_pixel_sizes=pps,
        channel_names=cn,
        channel_colors=channel_colors,
        scale_num_levels=4,
        scale_factor=2.0,
    )    

# here we are splitting multi-scene images into separate zarr images based on scene name
scene_indices = range(len(img.scenes))

for i in scene_indices:
    scenename = img.scenes[i]
    scenename = scenename.replace(":","_")
    write_scene(f"s3://{output_bucket}/{output_filename}/{scenename}.zarr/", scenename, i, img)


In [ ]:
#############################################
# READ BACK
#############################################
from ome_zarr.reader import Multiscales, Reader
from ome_zarr.io import parse_url

s3 = s3fs.S3FileSystem()
mypath = f"s3://{output_bucket}/{output_filename}/{scenes[scene_indices[0]]}.zarr" 

reader = Reader(parse_url(mypath))
node = list(reader())[0]
# levels
print(len(node.data))
for i in range(len(node.data)):
    print(f"shape of level {i} : {node.data[i].shape}")


In [ ]:
import nbvv
level = 1
levelxyscale = 2**(level+1)
t = 0
readdata = node.data[level][t][0:2].compute()
print(readdata.shape)
nbvv.volshow(readdata, spacing=(img.physical_pixel_sizes.X*levelxyscale,
        img.physical_pixel_sizes.Y*levelxyscale,
        img.physical_pixel_sizes.Z))